# 主題模型 Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

*<font color = blue>另一種流行的文本分析技術稱為主題建模。 主題建模的最終目標是找到語料庫中存在的各種主題。 語料庫中的每個文件將至少由一個主題（如果不是多個主題）組成。</font>*

*<font color = blue>在本筆記本中，我們將介紹如何進行潛在狄利克雷分配（LDA）的步驟，這是眾多主題建模技術之一。 它是專門為文字資料設計的。</font>*

*<font color = blue>若要使用主題建模技術，您需要提供 (1) 文件術語矩陣和 (2) 您希望演算法選取的主題數量。</font>*

*<font color = blue>應用主題建模技術後，您作為人類的工作是解釋結果並查看每個主題中的單字組合是否有意義。 如果它們沒有意義，您可以嘗試更改主題數量、文件術語矩陣中的術語、模型參數，甚至嘗試不同的模型。</font>*

## Topic Modeling - Attempt #1 (All Text)

In [1]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aaaaah,aaah,aah,abc,abcs,ability,abject,able,ablebodied,abortion,...,ze,zealand,zeppelin,zero,zillion,zombie,zombies,zoning,zoo,éclair
ali,0,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,2,...,0,10,0,0,0,0,0,0,0,0
bill,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,1,1,1,1,0,0
dave,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,1
louis,0,0,3,0,0,0,0,1,0,0,...,0,0,0,2,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,0,0,0,0,0
ricky,0,0,0,0,0,1,1,2,0,0,...,0,0,0,0,0,0,0,0,1,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,dave,jim,joe,john,louis,mike,ricky
aaaaah,0,0,1,0,0,0,0,0,0,0
aaah,0,0,0,1,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,3,0,0
abc,1,0,0,0,0,0,0,0,0,0
abcs,0,0,1,0,0,0,0,0,0,0


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
# 我們將把術語文檔矩陣放入新的 gensim 格式，從 df --> 稀疏矩陣 --> gensim 語料庫
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

*<font color = blue>現在我們有了語料庫（term-document matrix）和 id2word（dictionary of location: term），我們需要指定另外兩個參數 - 主題數量和傳遞次數。 讓我們從 2 開始主題數量，看看結果是否有意義，然後從那裡增加主題數量。</font>*

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
# 現在我們有了語料庫（term-document matrix）和 id2word（dictionary of location: term），
# 我們還需要指定另外兩個參數 - 主題數量和傳遞次數
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

2023-12-31 23:46:31,696 : INFO : using symmetric alpha at 0.5
2023-12-31 23:46:31,697 : INFO : using symmetric eta at 0.5
2023-12-31 23:46:31,700 : INFO : using serial LDA version on this node
2023-12-31 23:46:31,704 : INFO : running online (multi-pass) LDA training, 2 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:46:31,909 : INFO : -9.234 per-word bound, 602.2 perplexity estimate based on a held-out corpus of 10 documents with 82698 words
2023-12-31 23:46:31,910 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:46:31,949 : INFO : topic #0 (0.500): 0.038*"and" + 0.034*"the" + 0.030*"you" + 0.025*"to" + 0.014*"in" + 0.014*"of" + 0.013*"was" + 0.012*"that" + 0.011*"it" + 0.010*"my"
2023-12-31 23:46:31,951 : INFO : topic #1 (0.500): 0.036*"the" + 0.028*"and" + 0.024*"you" + 0.020*"that" + 0.019*"to" + 0.018*"of" + 0.

[(0,
  '0.039*"and" + 0.034*"the" + 0.023*"you" + 0.022*"to" + 0.016*"that" + 0.015*"was" + 0.015*"of" + 0.014*"in" + 0.012*"it" + 0.009*"my"'),
 (1,
  '0.036*"the" + 0.030*"and" + 0.028*"you" + 0.022*"to" + 0.017*"that" + 0.016*"of" + 0.015*"in" + 0.014*"it" + 0.011*"is" + 0.010*"was"')]

In [9]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

2023-12-29 13:45:00,055 : INFO : using symmetric alpha at 0.3333333333333333
2023-12-29 13:45:00,056 : INFO : using symmetric eta at 0.3333333333333333
2023-12-29 13:45:00,059 : INFO : using serial LDA version on this node
2023-12-29 13:45:00,063 : INFO : running online (multi-pass) LDA training, 3 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-29 13:45:00,260 : INFO : -9.362 per-word bound, 657.9 perplexity estimate based on a held-out corpus of 10 documents with 82755 words
2023-12-29 13:45:00,261 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-29 13:45:00,301 : INFO : topic #0 (0.333): 0.034*"the" + 0.031*"and" + 0.023*"to" + 0.022*"you" + 0.020*"of" + 0.016*"that" + 0.014*"it" + 0.013*"in" + 0.012*"was" + 0.010*"this"
2023-12-29 13:45:00,302 : INFO : topic #1 (0.333): 0.037*"and" + 0.030*"the" + 0.025*"you" + 0.024*"to" 

2023-12-29 13:45:01,944 : INFO : topic #0 (0.333): 0.032*"the" + 0.027*"you" + 0.026*"and" + 0.022*"to" + 0.018*"of" + 0.015*"that" + 0.013*"it" + 0.012*"in" + 0.012*"my" + 0.011*"is"
2023-12-29 13:45:01,946 : INFO : topic #1 (0.333): 0.041*"and" + 0.035*"the" + 0.021*"you" + 0.020*"to" + 0.016*"that" + 0.016*"was" + 0.014*"of" + 0.014*"in" + 0.014*"it" + 0.010*"my"
2023-12-29 13:45:01,947 : INFO : topic #2 (0.333): 0.037*"the" + 0.031*"you" + 0.030*"and" + 0.023*"to" + 0.018*"that" + 0.016*"of" + 0.016*"in" + 0.014*"it" + 0.011*"was" + 0.010*"is"
2023-12-29 13:45:01,948 : INFO : topic diff=0.022422, rho=0.301511
2023-12-29 13:45:01,949 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel<num_terms=6806, num_topics=3, decay=0.5, chunksize=2000> in 1.89s', 'datetime': '2023-12-29T13:45:01.949391', 'gensim': '4.3.0', 'python': '3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}
2023-12-29 13:45:01,951 : I

[(0,
  '0.032*"the" + 0.027*"you" + 0.026*"and" + 0.022*"to" + 0.018*"of" + 0.015*"that" + 0.013*"it" + 0.012*"in" + 0.012*"my" + 0.011*"is"'),
 (1,
  '0.041*"and" + 0.035*"the" + 0.021*"you" + 0.020*"to" + 0.016*"that" + 0.016*"was" + 0.014*"of" + 0.014*"in" + 0.014*"it" + 0.010*"my"'),
 (2,
  '0.037*"the" + 0.031*"you" + 0.030*"and" + 0.023*"to" + 0.018*"that" + 0.016*"of" + 0.016*"in" + 0.014*"it" + 0.011*"was" + 0.010*"is"')]

In [10]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

2023-12-29 13:45:09,320 : INFO : using symmetric alpha at 0.25
2023-12-29 13:45:09,321 : INFO : using symmetric eta at 0.25
2023-12-29 13:45:09,324 : INFO : using serial LDA version on this node
2023-12-29 13:45:09,328 : INFO : running online (multi-pass) LDA training, 4 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-29 13:45:09,486 : INFO : -9.445 per-word bound, 697.1 perplexity estimate based on a held-out corpus of 10 documents with 82755 words
2023-12-29 13:45:09,487 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-29 13:45:09,521 : INFO : topic #0 (0.250): 0.032*"and" + 0.029*"the" + 0.021*"you" + 0.017*"was" + 0.015*"of" + 0.014*"that" + 0.013*"in" + 0.013*"to" + 0.011*"it" + 0.009*"my"
2023-12-29 13:45:09,523 : INFO : topic #1 (0.250): 0.037*"the" + 0.035*"and" + 0.029*"you" + 0.025*"to" + 0.018*"of" + 0.017*"that" + 

2023-12-29 13:45:10,571 : INFO : topic #3 (0.250): 0.005*"the" + 0.004*"and" + 0.003*"you" + 0.002*"to" + 0.002*"that" + 0.002*"in" + 0.002*"it" + 0.002*"was" + 0.002*"of" + 0.001*"is"
2023-12-29 13:45:10,572 : INFO : topic diff=0.039244, rho=0.333333
2023-12-29 13:45:10,691 : INFO : -6.638 per-word bound, 99.6 perplexity estimate based on a held-out corpus of 10 documents with 82755 words
2023-12-29 13:45:10,692 : INFO : PROGRESS: pass 8, at document #10/10
2023-12-29 13:45:10,712 : INFO : topic #0 (0.250): 0.002*"and" + 0.002*"the" + 0.002*"you" + 0.001*"was" + 0.001*"of" + 0.001*"that" + 0.001*"in" + 0.001*"to" + 0.001*"it" + 0.001*"my"
2023-12-29 13:45:10,714 : INFO : topic #1 (0.250): 0.037*"the" + 0.035*"and" + 0.028*"you" + 0.023*"to" + 0.018*"that" + 0.017*"of" + 0.015*"in" + 0.015*"it" + 0.013*"was" + 0.011*"my"
2023-12-29 13:45:10,715 : INFO : topic #2 (0.250): 0.001*"the" + 0.001*"and" + 0.001*"you" + 0.001*"that" + 0.001*"to" + 0.001*"it" + 0.001*"of" + 0.001*"is" + 0.001*"

[(0,
  '0.002*"and" + 0.002*"the" + 0.001*"you" + 0.001*"was" + 0.001*"of" + 0.001*"that" + 0.001*"in" + 0.001*"to" + 0.001*"it" + 0.001*"my"'),
 (1,
  '0.037*"the" + 0.035*"and" + 0.028*"you" + 0.023*"to" + 0.018*"that" + 0.017*"of" + 0.015*"in" + 0.015*"it" + 0.013*"was" + 0.011*"my"'),
 (2,
  '0.001*"the" + 0.001*"and" + 0.001*"you" + 0.001*"that" + 0.001*"to" + 0.001*"it" + 0.001*"of" + 0.000*"is" + 0.000*"was" + 0.000*"my"'),
 (3,
  '0.003*"the" + 0.002*"and" + 0.002*"you" + 0.001*"to" + 0.001*"that" + 0.001*"in" + 0.001*"it" + 0.001*"was" + 0.001*"of" + 0.001*"is"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

*<font color = blue>這些主題看起來不太好。 我們嘗試修改參數。 我們也嘗試修改我們的術語清單。</font>*

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

*<font color = blue>一種流行的技巧是只查看來自一種詞性的術語（僅名詞、僅形容詞等）。 查看賓州大學標籤集：</font>*

In [7]:
# Let's create a function to pull out nouns from a string of text
# 讓我們創建一個函數來從文字字串中提取名詞
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [8]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
ali,ladies and gentlemen please welcome to the sta...
anthony,thank you thank you thank you san francisco th...
bill,all right thank you thank you very much thank...
dave,this is dave he tells dirty jokes for a living...
jim,ladies and gentlemen please welcome to the ...
joe,ladies and gentlemen welcome joe rogan wha...
john,armed with boyish charm and a sharp wit the fo...
louis,intro fade the music out lets roll hold there ...
mike,wow hey thank you thanks thank you guys hey se...
ricky,hello hello how you doing great thank you wow ...


In [12]:
# 沒有'averaged_perceptron_tagger' 因此要下載
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daocheng/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\daocheng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [34]:
# Apply the nouns function to the transcripts to filter only on nouns
# 將名詞函數應用於轉錄本以僅過濾名詞
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ladies gentlemen stage ali hi thank hello na s...
anthony,thank thank people i em i francisco city world...
bill,thank thank pleasure georgia area oasis i june...
dave,jokes living stare work profound train thought...
jim,ladies gentlemen stage mr jim jefferies thank ...
joe,ladies gentlemen joe fck thanks phone fckface ...
john,charm wit snl writer john mulaney marriage bee...
louis,intro music lets lights lights thank i i place...
mike,wow hey thanks look insane years everyone i id...
ricky,hello thank fuck thank im gon youre weve money...


In [14]:
# Create a new document-term matrix using only nouns
# 僅使用名詞建立新的文檔術語矩陣
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=list(stop_words))
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aah,abc,abcs,ability,abortion,abortions,abuse,accent,accents,access,...,yulin,yummy,ze,zealand,zeppelin,zillion,zombie,zombies,zoo,éclair
ali,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,2,0,0,1,0,0,...,0,0,0,10,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,1,0,0
dave,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,1
louis,3,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
ricky,0,0,0,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0


In [15]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [16]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

2023-12-31 23:52:37,625 : INFO : using symmetric alpha at 0.5
2023-12-31 23:52:37,627 : INFO : using symmetric eta at 0.5
2023-12-31 23:52:37,629 : INFO : using serial LDA version on this node
2023-12-31 23:52:37,633 : INFO : running online (multi-pass) LDA training, 2 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:52:37,748 : INFO : -8.806 per-word bound, 447.5 perplexity estimate based on a held-out corpus of 10 documents with 16907 words
2023-12-31 23:52:37,749 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:52:37,782 : INFO : topic #0 (0.500): 0.009*"day" + 0.009*"thing" + 0.008*"shit" + 0.007*"cause" + 0.007*"hes" + 0.006*"man" + 0.005*"life" + 0.005*"guy" + 0.005*"theyre" + 0.005*"years"
2023-12-31 23:52:37,783 : INFO : topic #1 (0.500): 0.009*"thing" + 0.008*"shit" + 0.008*"day" + 0.008*"way" + 0.008*"man"

[(0,
  '0.011*"shit" + 0.010*"day" + 0.008*"thing" + 0.007*"guy" + 0.007*"gon" + 0.006*"fuck" + 0.006*"hes" + 0.006*"cause" + 0.006*"women" + 0.006*"man"'),
 (1,
  '0.009*"thing" + 0.008*"day" + 0.008*"cause" + 0.008*"way" + 0.007*"life" + 0.007*"man" + 0.007*"things" + 0.007*"shit" + 0.007*"hes" + 0.006*"guy"')]

In [17]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

2023-12-31 23:53:00,992 : INFO : using symmetric alpha at 0.3333333333333333
2023-12-31 23:53:00,994 : INFO : using symmetric eta at 0.3333333333333333
2023-12-31 23:53:00,996 : INFO : using serial LDA version on this node
2023-12-31 23:53:01,000 : INFO : running online (multi-pass) LDA training, 3 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:53:01,111 : INFO : -9.029 per-word bound, 522.2 perplexity estimate based on a held-out corpus of 10 documents with 16907 words
2023-12-31 23:53:01,111 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:53:01,150 : INFO : topic #0 (0.333): 0.010*"thing" + 0.008*"day" + 0.007*"cause" + 0.007*"hes" + 0.007*"shit" + 0.006*"gon" + 0.006*"life" + 0.006*"guy" + 0.005*"years" + 0.005*"kids"
2023-12-31 23:53:01,152 : INFO : topic #1 (0.333): 0.010*"day" + 0.009*"shit" + 0.009*"man" +

[(0,
  '0.001*"thing" + 0.001*"day" + 0.001*"cause" + 0.001*"hes" + 0.001*"years" + 0.001*"life" + 0.001*"kids" + 0.001*"gon" + 0.001*"night" + 0.001*"way"'),
 (1,
  '0.012*"shit" + 0.010*"thing" + 0.009*"cause" + 0.009*"man" + 0.008*"life" + 0.008*"day" + 0.008*"guy" + 0.008*"hes" + 0.008*"gon" + 0.007*"lot"'),
 (2,
  '0.011*"day" + 0.010*"joke" + 0.009*"thing" + 0.007*"way" + 0.006*"years" + 0.006*"things" + 0.005*"hes" + 0.005*"mom" + 0.005*"dad" + 0.005*"baby"')]

In [18]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

2023-12-31 23:53:04,163 : INFO : using symmetric alpha at 0.25
2023-12-31 23:53:04,165 : INFO : using symmetric eta at 0.25
2023-12-31 23:53:04,167 : INFO : using serial LDA version on this node
2023-12-31 23:53:04,172 : INFO : running online (multi-pass) LDA training, 4 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:53:04,296 : INFO : -9.293 per-word bound, 627.3 perplexity estimate based on a held-out corpus of 10 documents with 16907 words
2023-12-31 23:53:04,296 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:53:04,328 : INFO : topic #0 (0.250): 0.011*"thing" + 0.007*"cause" + 0.007*"life" + 0.006*"hes" + 0.006*"shit" + 0.006*"guy" + 0.005*"day" + 0.005*"man" + 0.005*"things" + 0.005*"kids"
2023-12-31 23:53:04,330 : INFO : topic #1 (0.250): 0.011*"shit" + 0.008*"thing" + 0.007*"day" + 0.007*"gon" + 0.007*"guy

[(0,
  '0.017*"life" + 0.013*"thing" + 0.010*"hes" + 0.009*"cause" + 0.008*"kids" + 0.008*"course" + 0.007*"tit" + 0.007*"guy" + 0.006*"shit" + 0.006*"way"'),
 (1,
  '0.009*"shit" + 0.009*"gon" + 0.009*"guy" + 0.009*"cause" + 0.008*"thing" + 0.007*"kind" + 0.007*"hes" + 0.007*"way" + 0.007*"point" + 0.006*"day"'),
 (2,
  '0.011*"shit" + 0.010*"day" + 0.009*"lot" + 0.008*"man" + 0.007*"way" + 0.007*"thing" + 0.006*"mom" + 0.006*"cause" + 0.006*"women" + 0.005*"money"'),
 (3,
  '0.011*"thing" + 0.011*"day" + 0.009*"man" + 0.007*"things" + 0.007*"cause" + 0.007*"fck" + 0.007*"life" + 0.007*"hes" + 0.007*"theyre" + 0.006*"house"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [19]:
# Let's create a function to pull out nouns from a string of text
# 讓我們創建一個函數來從文字字串中提取名詞
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [20]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ladies gentlemen welcome stage ali wong hi wel...
anthony,thank san francisco thank good people surprise...
bill,right thank thank pleasure greater atlanta geo...
dave,dirty jokes living stare most hard work profou...
jim,ladies gentlemen welcome stage mr jim jefferie...
joe,ladies gentlemen joe fck san francisco thanks ...
john,boyish charm sharp wit former snl writer john ...
louis,intro music lets lights lights thank much i i ...
mike,wow hey thanks hey seattle nice look crazy ins...
ricky,hello great thank fuck thank lovely welcome im...


In [21]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
# 僅使用名詞和形容詞建立新的文件術語矩陣，並使用 max_df 刪除常見單字
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aah,abc,abcs,ability,abject,able,ablebodied,abortion,abortions,...,yummy,yyou,ze,zealand,zeppelin,zillion,zombie,zombies,zoo,éclair
ali,0,0,1,0,0,0,2,0,0,0,...,0,1,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,0,2,0,...,0,0,0,10,0,0,0,0,0,0
bill,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,1,1,0,0
dave,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
louis,0,3,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
ricky,0,0,0,0,1,1,2,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [22]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [23]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

2023-12-31 23:53:17,928 : INFO : using symmetric alpha at 0.5
2023-12-31 23:53:17,930 : INFO : using symmetric eta at 0.5
2023-12-31 23:53:17,933 : INFO : using serial LDA version on this node
2023-12-31 23:53:17,936 : INFO : running online (multi-pass) LDA training, 2 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:53:18,036 : INFO : -9.031 per-word bound, 523.3 perplexity estimate based on a held-out corpus of 10 documents with 15674 words
2023-12-31 23:53:18,036 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:53:18,082 : INFO : topic #0 (0.500): 0.005*"joke" + 0.005*"fuck" + 0.004*"kid" + 0.004*"dude" + 0.003*"fucking" + 0.003*"mom" + 0.003*"different" + 0.003*"everybody" + 0.003*"men" + 0.003*"dead"
2023-12-31 23:53:18,083 : INFO : topic #1 (0.500): 0.005*"fuck" + 0.005*"fucking" + 0.004*"kid" + 0.004*"dude" +

[(0,
  '0.006*"fuck" + 0.006*"joke" + 0.005*"fucking" + 0.004*"dude" + 0.004*"kid" + 0.003*"everybody" + 0.003*"dead" + 0.003*"jenny" + 0.002*"okay" + 0.002*"sure"'),
 (1,
  '0.006*"fck" + 0.005*"kid" + 0.004*"different" + 0.004*"ass" + 0.004*"girl" + 0.003*"mom" + 0.003*"clinton" + 0.003*"fuck" + 0.003*"men" + 0.003*"dog"')]

In [24]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

2023-12-31 23:53:20,201 : INFO : using symmetric alpha at 0.3333333333333333
2023-12-31 23:53:20,202 : INFO : using symmetric eta at 0.3333333333333333
2023-12-31 23:53:20,205 : INFO : using serial LDA version on this node
2023-12-31 23:53:20,208 : INFO : running online (multi-pass) LDA training, 3 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:53:20,323 : INFO : -9.308 per-word bound, 633.8 perplexity estimate based on a held-out corpus of 10 documents with 15674 words
2023-12-31 23:53:20,323 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:53:20,376 : INFO : topic #0 (0.333): 0.004*"fuck" + 0.004*"kid" + 0.004*"dude" + 0.004*"joke" + 0.003*"mom" + 0.003*"fucking" + 0.003*"okay" + 0.003*"everybody" + 0.003*"friend" + 0.003*"parents"
2023-12-31 23:53:20,377 : INFO : topic #1 (0.333): 0.006*"fuck" + 0.005*"fucking"

[(0,
  '0.004*"mom" + 0.004*"ok" + 0.004*"jenny" + 0.004*"clinton" + 0.004*"parents" + 0.004*"kid" + 0.004*"uh" + 0.003*"friend" + 0.003*"course" + 0.003*"different"'),
 (1,
  '0.009*"joke" + 0.008*"fuck" + 0.007*"fucking" + 0.004*"men" + 0.004*"jokes" + 0.004*"guns" + 0.004*"son" + 0.003*"kid" + 0.003*"ahah" + 0.003*"anthony"'),
 (2,
  '0.010*"fck" + 0.009*"dude" + 0.006*"everybody" + 0.006*"kid" + 0.006*"fuck" + 0.005*"fucking" + 0.005*"fcking" + 0.004*"weird" + 0.004*"dick" + 0.004*"religion"')]

In [25]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

2023-12-31 23:53:21,589 : INFO : using symmetric alpha at 0.25
2023-12-31 23:53:21,589 : INFO : using symmetric eta at 0.25
2023-12-31 23:53:21,601 : INFO : using serial LDA version on this node
2023-12-31 23:53:21,606 : INFO : running online (multi-pass) LDA training, 4 topics, 10 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:53:21,746 : INFO : -9.638 per-word bound, 796.7 perplexity estimate based on a held-out corpus of 10 documents with 15674 words
2023-12-31 23:53:21,747 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:53:21,790 : INFO : topic #0 (0.250): 0.007*"fuck" + 0.005*"dude" + 0.005*"fucking" + 0.004*"joke" + 0.004*"kid" + 0.003*"mom" + 0.003*"everybody" + 0.003*"okay" + 0.002*"different" + 0.002*"men"
2023-12-31 23:53:21,792 : INFO : topic #1 (0.250): 0.005*"kid" + 0.005*"fucking" + 0.004*"fuck" + 0.004*"dude"

[(0,
  '0.010*"fuck" + 0.008*"dude" + 0.007*"fucking" + 0.006*"everybody" + 0.005*"ahah" + 0.004*"kid" + 0.004*"gay" + 0.004*"husband" + 0.003*"son" + 0.003*"men"'),
 (1,
  '0.000*"kid" + 0.000*"fucking" + 0.000*"fuck" + 0.000*"dude" + 0.000*"everybody" + 0.000*"ass" + 0.000*"mom" + 0.000*"different" + 0.000*"uh" + 0.000*"joke"'),
 (2,
  '0.005*"fucking" + 0.004*"fuck" + 0.004*"joke" + 0.004*"course" + 0.004*"jenny" + 0.003*"different" + 0.003*"dog" + 0.003*"ass" + 0.003*"guns" + 0.003*"kid"'),
 (3,
  '0.009*"fck" + 0.007*"mom" + 0.006*"kid" + 0.006*"joke" + 0.005*"clinton" + 0.005*"okay" + 0.005*"fcking" + 0.004*"anthony" + 0.004*"parents" + 0.004*"president"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

*<font color = blue>在我們查看的 9 個主題模型（名詞和形容詞）中，有 4 個主題模型最有意義。 因此，讓我們將其放在這裡​​並通過更多迭代來運行它以獲得更多微調的主題。</font>*

In [26]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

2023-12-31 23:53:25,266 : INFO : using symmetric alpha at 0.25
2023-12-31 23:53:25,267 : INFO : using symmetric eta at 0.25
2023-12-31 23:53:25,271 : INFO : using serial LDA version on this node
2023-12-31 23:53:25,275 : INFO : running online (multi-pass) LDA training, 4 topics, 80 passes over the supplied corpus of 10 documents, updating model once every 10 documents, evaluating perplexity every 10 documents, iterating 50x with a convergence threshold of 0.001000
2023-12-31 23:53:25,510 : INFO : -9.635 per-word bound, 794.9 perplexity estimate based on a held-out corpus of 10 documents with 15674 words
2023-12-31 23:53:25,510 : INFO : PROGRESS: pass 0, at document #10/10
2023-12-31 23:53:25,554 : INFO : topic #0 (0.250): 0.005*"joke" + 0.005*"fck" + 0.004*"kid" + 0.004*"fuck" + 0.004*"dude" + 0.004*"different" + 0.004*"okay" + 0.004*"everybody" + 0.003*"girl" + 0.003*"mom"
2023-12-31 23:53:25,554 : INFO : topic #1 (0.250): 0.007*"fuck" + 0.007*"fucking" + 0.005*"kid" + 0.004*"dude" + 

[(0,
  '0.010*"fck" + 0.007*"joke" + 0.006*"dude" + 0.006*"kid" + 0.005*"mom" + 0.005*"everybody" + 0.005*"fcking" + 0.005*"anthony" + 0.004*"husband" + 0.004*"girl"'),
 (1,
  '0.008*"fucking" + 0.008*"fuck" + 0.006*"kid" + 0.005*"joke" + 0.004*"dude" + 0.004*"gun" + 0.004*"course" + 0.004*"ass" + 0.004*"dog" + 0.004*"guns"'),
 (2,
  '0.007*"clinton" + 0.007*"jenny" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"'),
 (3,
  '0.010*"ahah" + 0.009*"fuck" + 0.007*"gay" + 0.005*"fucking" + 0.004*"son" + 0.004*"oj" + 0.004*"ghetto" + 0.004*"men" + 0.004*"young" + 0.004*"motherfucker"')]

2023-12-29 14:18:55,183 : INFO : PROGRESS: pass 29, at document #10/10
2023-12-29 14:18:55,197 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"fucking" + 0.004*"jokes" + 0.004*"son"
2023-12-29 14:18:55,198 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"kid" + 0.006*"fucking" + 0.006*"fck" + 0.005*"everybody" + 0.005*"ass" + 0.004*"different" + 0.004*"men" + 0.004*"girl"
2023-12-29 14:18:55,200 : INFO : topic #2 (0.250): 0.007*"clinton" + 0.007*"jenny" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"
2023-12-29 14:18:55,200 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"hampstead" + 0.005*"chimp" + 0.005*"rape" + 0.004*"fucking" + 0.004*"funny" + 0.004*"twitter" + 0.004*"tweet"
2023-12-29 14:18:55,201 : INFO : topic diff=0.000106, rho=0.179605
2023-12-29 14:18:55,267 : 

2023-12-29 14:18:55,830 : INFO : topic #2 (0.250): 0.007*"clinton" + 0.007*"jenny" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"
2023-12-29 14:18:55,831 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"hampstead" + 0.005*"chimp" + 0.005*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"
2023-12-29 14:18:55,832 : INFO : topic diff=0.000026, rho=0.162221
2023-12-29 14:18:55,902 : INFO : -8.032 per-word bound, 261.8 perplexity estimate based on a held-out corpus of 10 documents with 15691 words
2023-12-29 14:18:55,903 : INFO : PROGRESS: pass 37, at document #10/10
2023-12-29 14:18:55,919 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"fucking" + 0.004*"jokes" + 0.004*"son"
2023-12-29 14:18:55,920 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"kid"

2023-12-29 14:18:56,436 : INFO : topic diff=0.000008, rho=0.149071
2023-12-29 14:18:56,520 : INFO : -8.032 per-word bound, 261.8 perplexity estimate based on a held-out corpus of 10 documents with 15691 words
2023-12-29 14:18:56,522 : INFO : PROGRESS: pass 44, at document #10/10
2023-12-29 14:18:56,533 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"fucking" + 0.004*"jokes" + 0.004*"son"
2023-12-29 14:18:56,534 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"kid" + 0.006*"fucking" + 0.006*"fck" + 0.005*"everybody" + 0.005*"ass" + 0.004*"men" + 0.004*"different" + 0.004*"girl"
2023-12-29 14:18:56,536 : INFO : topic #2 (0.250): 0.007*"clinton" + 0.007*"jenny" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"
2023-12-29 14:18:56,537 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0

2023-12-29 14:18:57,239 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"kid" + 0.006*"fucking" + 0.006*"fck" + 0.005*"everybody" + 0.005*"ass" + 0.004*"men" + 0.004*"different" + 0.004*"girl"
2023-12-29 14:18:57,241 : INFO : topic #2 (0.250): 0.007*"clinton" + 0.007*"jenny" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"
2023-12-29 14:18:57,242 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"hampstead" + 0.005*"chimp" + 0.005*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"
2023-12-29 14:18:57,243 : INFO : topic diff=0.000002, rho=0.137361
2023-12-29 14:18:57,318 : INFO : -8.032 per-word bound, 261.8 perplexity estimate based on a held-out corpus of 10 documents with 15691 words
2023-12-29 14:18:57,319 : INFO : PROGRESS: pass 52, at document #10/10
2023-12-29 14:18:57,330 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ah

2023-12-29 14:18:57,973 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"hampstead" + 0.005*"chimp" + 0.005*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"
2023-12-29 14:18:57,974 : INFO : topic diff=0.000001, rho=0.129099
2023-12-29 14:18:58,039 : INFO : -8.032 per-word bound, 261.8 perplexity estimate based on a held-out corpus of 10 documents with 15691 words
2023-12-29 14:18:58,040 : INFO : PROGRESS: pass 59, at document #10/10
2023-12-29 14:18:58,054 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"jokes" + 0.004*"fucking" + 0.004*"shark"
2023-12-29 14:18:58,056 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"kid" + 0.006*"fucking" + 0.006*"fck" + 0.005*"everybody" + 0.005*"ass" + 0.004*"men" + 0.004*"different" + 0.004*"girl"
2023-12-29 14:18:58,057 : INFO : topic #2 (0.250): 0.007*"clinton" + 0.007*"jenny" + 0.00

2023-12-29 14:18:58,703 : INFO : PROGRESS: pass 66, at document #10/10
2023-12-29 14:18:58,718 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"jokes" + 0.004*"fucking" + 0.004*"shark"
2023-12-29 14:18:58,719 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"kid" + 0.006*"fucking" + 0.006*"fck" + 0.005*"everybody" + 0.005*"ass" + 0.004*"men" + 0.004*"different" + 0.004*"girl"
2023-12-29 14:18:58,719 : INFO : topic #2 (0.250): 0.007*"clinton" + 0.007*"jenny" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"
2023-12-29 14:18:58,721 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"chimp" + 0.005*"hampstead" + 0.005*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"
2023-12-29 14:18:58,722 : INFO : topic diff=0.000000, rho=0.121268
2023-12-29 14:18:58,786 

2023-12-29 14:18:59,391 : INFO : topic #2 (0.250): 0.007*"jenny" + 0.007*"clinton" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"
2023-12-29 14:18:59,392 : INFO : topic #3 (0.250): 0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"chimp" + 0.005*"hampstead" + 0.005*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"
2023-12-29 14:18:59,393 : INFO : topic diff=0.000000, rho=0.115470
2023-12-29 14:18:59,464 : INFO : -8.032 per-word bound, 261.8 perplexity estimate based on a held-out corpus of 10 documents with 15691 words
2023-12-29 14:18:59,465 : INFO : PROGRESS: pass 74, at document #10/10
2023-12-29 14:18:59,480 : INFO : topic #0 (0.250): 0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"jokes" + 0.004*"fucking" + 0.004*"shark"
2023-12-29 14:18:59,482 : INFO : topic #1 (0.250): 0.007*"dude" + 0.007*"fuck" + 0.006*"ki

[(0,
  '0.009*"joke" + 0.008*"fuck" + 0.007*"ahah" + 0.007*"anthony" + 0.006*"mad" + 0.005*"gay" + 0.004*"grandma" + 0.004*"jokes" + 0.004*"fucking" + 0.004*"shark"'),
 (1,
  '0.007*"dude" + 0.007*"fuck" + 0.006*"kid" + 0.006*"fucking" + 0.006*"fck" + 0.005*"everybody" + 0.005*"ass" + 0.004*"men" + 0.004*"different" + 0.004*"girl"'),
 (2,
  '0.007*"jenny" + 0.007*"clinton" + 0.005*"parents" + 0.005*"friend" + 0.005*"mom" + 0.004*"cow" + 0.004*"wife" + 0.004*"john" + 0.004*"phone" + 0.004*"sure"'),
 (3,
  '0.010*"joke" + 0.006*"nuts" + 0.005*"jenner" + 0.005*"hampstead" + 0.005*"chimp" + 0.005*"rape" + 0.004*"funny" + 0.004*"fucking" + 0.004*"twitter" + 0.004*"tweet"')]

These four topics look pretty decent. Let's settle on these for now.

*<font color = blue>這四個題目看起來都還不錯。 讓我們暫時解決這些問題。</font>*

* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [27]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(0, 'ali'),
 (0, 'anthony'),
 (1, 'bill'),
 (3, 'dave'),
 (1, 'jim'),
 (0, 'joe'),
 (2, 'john'),
 (1, 'louis'),
 (2, 'mike'),
 (1, 'ricky')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]

## Additional Exercises

1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.